In [3]:
import pandas as pd

In [11]:
df = pd.read_csv('/dataset_with_sentiments.csv')

In [12]:
df

,Name,Comment,Date,Model,sentiment
0,Mazda 3s,After months of releasing preview images and s...,"August 29, 2022",Maserati Grecale,3
1,Darepoole,It looks like the Macan with a little bit of I...,"March 22, 2022",Maserati Grecale,2
2,JackStraw79,Nice hot hatch!,"March 22, 2022",Maserati Grecale,4
3,CostcoPizza,"Looks like a car you'd find in GTA V DLC, whic...","March 22, 2022",Maserati Grecale,3
4,Venom21,Terrible gloss black: CheckTerrible touchscree...,"March 22, 2022",Maserati Grecale,0
...,...,...,...,...,...
49370,Wing Commander,I suppose we should be grateful that at least ...,"October 02, 2021",Porsche Macan,4
49371,sd1985,Our first macan was a base on 21s with no Pasm...,"October 02, 2021",Porsche Macan,2
49372,BanZ,"For gen3, on the uk configurator PASM is not c...","October 02, 2021",Porsche Macan,3
49373,Col Lamb,Well that is another reason for avoiding the 2...,"October 02, 2021",Porsche Macan,0


In [13]:
# Classify comments based on sentiment values
df['sentiment_label'] = df['sentiment'].apply(lambda x: 'Positive' if x > 2 else ('Negative' if x < 2 else 'Neutral'))

# Group by Model and sentiment_label to get counts
sentiment_counts = df.groupby(['Model', 'sentiment_label']).size().unstack().reset_index()

sentiment_counts


sentiment_label,Model,Negative,Neutral,Positive
0,Alfa Romeo Stelvio,3147,2551,2759
1,Audi Q5,3984,2743,3271
2,BMW X3,4671,3086,4276
3,Maserati Grecale,373,216,298
4,Porsche Macan,6676,5122,6202


In [16]:
# Calculate the percentage for each sentiment label based on the total comments for each car model
sentiment_percentages = sentiment_counts.set_index('Model').div(sentiment_counts.set_index('Model').sum(axis=1), axis=0) * 100


In [17]:
sentiment_percentages

sentiment_label,Negative,Neutral,Positive
Model,,,
Alfa Romeo Stelvio,37.211777,30.164361,32.623862
Audi Q5,39.847970,27.435487,32.716543
BMW X3,38.818250,25.646140,35.535610
Maserati Grecale,42.051860,24.351747,33.596392
Porsche Macan,37.088889,28.455556,34.455556


In [6]:
df2 = pd.read_csv('/CombinedPistonheads.csv')

In [7]:
df2

,Name,Comment,Date,Model
0,Gumpclipper,"Good morning all,I am looking at buying my fir...","October 10, 2021",Porsche Macan
1,Voodoo Blue,I’m not going to offer an opinion on spec as t...,"October 10, 2021",Porsche Macan
2,Gumpclipper,Fantastic info. Thank you! Will take a look at...,"October 10, 2021",Porsche Macan
3,TheBMWDriver,Right now 1% is best you can get on the macan....,"October 21, 2021",Porsche Macan
4,neilf,My tip would be to get your name on the deposi...,"October 22, 2021",Porsche Macan
...,...,...,...,...
1392,urquattroGus,SWoll said:So based on the poster in question ...,"March 28, 2022",Maserati Grecale
1393,1974foggy,Hyundai and Kia make better looking SUVs,"March 30, 2022",Maserati Grecale
1394,ettore,This issue here is that it’s exceptionally dul...,"March 30, 2022",Maserati Grecale
1395,CDP,True.The car isn't ugly by any means; it's com...,"March 31, 2022",Maserati Grecale


In [8]:
import numpy as np

In [9]:
!pip install transformers torch
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, random_split
from transformers import BertTokenizer, BertForSequenceClassification,AdamW
from torch.nn.functional import softmax
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.2 MB/s eta 0:00:00


In [10]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment" # This BERT model is trained for sentiment analysis.
model = BertForSequenceClassification.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [11]:
def predict_sentiments(texts, batch_size):
    sentiments = []
    model.to(device)  # Ensure model is on the correct device

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
        with torch.no_grad():
            logits = model(**inputs).logits
        probs = softmax(logits, dim=-1)
        preds = torch.argmax(probs, dim=1)
        sentiments.extend(preds.cpu().numpy())
    return sentiments

In [ ]:
df2['Comment'] = df2['Comment'].astype(str)
batch_size = 32
df2['sentiment'] = predict_sentiments(df2['Comment'].tolist(), batch_size)